In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# === PART 1: Test single image prediction ===
path = '../clothing-dataset-small/train/t-shirt'
name = '0a85a584-cb49-4795-b2f1-7eebbf09399a.jpg'
fullname = f'{path}/{name}'

img = load_img(fullname, target_size=(299, 299))
x = np.array(img)
X = preprocess_input(x)
X = np.expand_dims(X, axis=0)

model = Xception(weights='imagenet', input_shape=(299, 299, 3))
pred = model.predict(X)

# === PART 2: Create data generators ===
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_gen.flow_from_directory(
    '../clothing-dataset-small/train', 
    target_size=(299, 299),  # Changed to 299
    batch_size=32,
    class_mode='categorical'
)

val_ds = val_gen.flow_from_directory(
    '../clothing-dataset-small/validation', 
    target_size=(299, 299),  # Changed to 299
    batch_size=32,
    shuffle=False,
    class_mode='categorical'
)

# === PART 3: Create custom model for transfer learning ===
base_model = Xception(
    weights='imagenet', 
    include_top=False,  # Remove the classification layer
    input_shape=(299, 299, 3)  # Changed to 299
)
base_model.trainable = False  # Freeze the base model

# Add your own layers on top
inputs = keras.Input(shape=(299, 299, 3))  # Changed to 299
base = base_model(inputs, training=False)

# Add pooling and classification layers
vectors = keras.layers.GlobalAveragePooling2D()(base)
x = keras.layers.Dense(3, activation='relu')(vectors)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(train_ds.num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)


# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())